# Creating and Backtesting RSI Strategies

## Getting the Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [ ]:
data = pd.read_csv("eurusd.csv", parse_dates = ["Date"], index_col = "Date")

In [ ]:
data

In [ ]:
data.info()

In [ ]:
data.plot(figsize = (12, 8), title = "EUR/USD", fontsize = 12)
plt.show()

## Defining an RSI Strategy (Part 1)

In [ ]:
data

In [ ]:
data.price.diff()

__U:__ <br>
- if __positive__ price change ("up"): __price change__<br>
- if __negative__ price change ("down"): __0__

In [ ]:
data["U"] = np.where(data.price.diff() > 0, data.price.diff(), 0)

__D:__ <br>
- if __positive__ price change ("up"): __0__<br>
- if __negative__ price change ("down"): __-price change__

In [ ]:
data["D"] = np.where(data.price.diff() < 0, -data.price.diff(), 0)

In [ ]:
data

__Calculate Moving Averages for U and D__ (alternative: exponential smoothing)

In [ ]:
periods = 20

In [ ]:
data["MA_U"] = data.U.rolling(periods).mean()

In [ ]:
data["MA_D"] = data.D.rolling(periods).mean()

In [ ]:
data

__Calculate Relative Strength Index (RSI)__

Ratio of ups to downs

In [ ]:
data["RSI"] = data.MA_U / (data.MA_U + data.MA_D) * 100

In [ ]:
data.dropna(inplace = True)

In [ ]:
data

In [ ]:
data.loc["2019", ["price", "RSI"]].plot(figsize = (12, 8), secondary_y = "RSI")
plt.show()

Instruments which have had
- __more or stronger positive changes__ have a __higher RSI__
- __more or stronger negative changes__ have a __lower RSI__

## Defining an RSI Strategy (Part 2)

- Extreme __high RSI__ (>70) indicates __overbought__ instrument -> __sell__
- Extreme __low RSI__ (<30) indicates __oversold__ instrument -> __buy__ 

In [ ]:
rsi_upper = 70
rsi_lower = 30

In [ ]:
data.loc["2010", ["RSI"]].plot(figsize = (12, 8))
plt.hlines(y = rsi_upper, xmin = data.index[0], xmax = data.index[-1], label = "RSI_Upper", color = "r")
plt.hlines(y = rsi_lower, xmin = data.index[0], xmax = data.index[-1], label = "RSI_Lower", color = "g")
plt.yticks(ticks = range(0, 100, 10))
plt.legend()
plt.show()

In [ ]:
data["position"] = np.where(data.RSI > rsi_upper, -1, np.nan) # 1. overbought -> go short

In [ ]:
data["position"] = np.where(data.RSI < rsi_lower, 1, data.position) # 2. oversold -> go long

In [ ]:
data.position = data.position.fillna(0) # 3. where 1 & 2 isn´t applicable -> neutral

In [ ]:
data

In [ ]:
data.position.value_counts()

In [ ]:
data.loc[:, ["RSI", "position"]].plot(figsize = (12, 8), fontsize = 12, 
                                                      secondary_y = "position", title = "EUR/USD")
plt.show()

In [ ]:
data.loc["06-2010":"07-2010", ["RSI", "position"]].plot(figsize = (12, 8), fontsize = 12, 
                                                      secondary_y = "position", title = "EUR/USD")
plt.show()

## Vectorized Strategy Backtesting

In [ ]:
data

In [ ]:
data["returns"] = np.log(data.price.div(data.price.shift(1)))

In [ ]:
data.dropna(inplace = True)

In [ ]:
data

In [ ]:
data["strategy"] = data.position.shift(1) * data["returns"]

In [ ]:
data

In [ ]:
data.dropna(inplace = True)

In [ ]:
ptc = 0.00007

In [ ]:
data["trades"] = data.position.diff().fillna(0).abs()

In [ ]:
data.trades.value_counts()

In [ ]:
data["strategy_net"] = data.strategy - data.trades * ptc

In [ ]:
data["creturns"] = data["returns"].cumsum().apply(np.exp)
data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
data["cstrategy_net"] = data["strategy_net"].cumsum().apply(np.exp)

In [ ]:
data

In [ ]:
data[["creturns", "cstrategy", "cstrategy_net"]].plot(figsize = (12 , 8))
plt.show()

In [ ]:
data.loc["2020", ["creturns", "cstrategy", "cstrategy_net"]].plot(figsize = (12 , 8))
plt.show()

## Using the RSIBacktester Class

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brute
plt.style.use("seaborn")

In [ ]:
pd.read_csv("forex_pairs.csv")

In [ ]:
from RSIBacktester import RSIBacktester as RSI

In [ ]:
ptc = 0.00007

In [ ]:
tester = RSI("EURUSD=X", periods = 20, rsi_upper = 70, rsi_lower = 30,
              start = "2004-01-01", end = "2020-06-30", tc = ptc)

In [ ]:
tester

In [ ]:
tester.data

In [ ]:
tester.test_strategy()

In [ ]:
tester.plot_results()

In [ ]:
tester.results

In [ ]:
tester.optimize_parameters((5, 20, 1), (65, 80, 1), (20, 35, 1)) # higher volatility

In [ ]:
tester.plot_results()

In [ ]:
tester.optimize_parameters((20, 50, 1), (75, 85, 1), (15, 25, 1)) # lower volatility

In [ ]:
tester.plot_results()

__USD / GBP__

In [ ]:
tester = RSI("USDGBP=X", periods = 20, rsi_upper = 70, rsi_lower = 30,
              start = "2004-01-01", end = "2020-06-30", tc = 0)

In [ ]:
tester.test_strategy()

In [ ]:
tester.plot_results()

In [ ]:
tester.optimize_parameters((5, 20, 1), (65, 80, 1), (20, 35, 1)) # high volatility

In [ ]:
tester.plot_results()

## Alternative Strategies and Interpretations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brute
plt.style.use("seaborn")
from RSIBacktester import RSIBacktester as RSI

In [ ]:
tester = RSI("EURUSD=X", periods = 20, rsi_upper = 70, rsi_lower = 30,
              start = "2004-01-01", end = "2020-06-30", tc = 0)

In [ ]:
alt = tester.data.dropna()
alt

Convergence: 
- if __Price__ reaches __higher highs__ -> __RSI__ reaches __higher highs__ (confirmation)
- if __Price__ reaches __lower lows__ -> __RSI__ reaches __lower lows__ (confirmation)

In [ ]:
alt.loc["03-2013":"06-2013", ["price", "RSI"]].plot(figsize = (12, 8), secondary_y = "RSI")
plt.show()

Divergence: 
- if __Price__ reaches __higher highs__ -> __RSI fails__ to reach higher highs (__Bearish Divergence__)
- if __Price__ reaches __lower lows__ -> __RSI fails__ to reach lower lows (__Bullish Divergence__)

In [ ]:
alt.loc["03-2016":"06-2016", ["price", "RSI"]].plot(figsize = (12, 8), secondary_y = "RSI")
plt.title("Bearish Divergence", fontsize = 15)
plt.show()

In [ ]:
alt.loc["05-2012":"11-2012", ["price", "RSI"]].plot(figsize = (12, 8), secondary_y = "RSI")
plt.title("Bullish Divergence", fontsize = 15)
plt.show()